Importing the necessary libraries

In [1]:
import pandas as pd
import bigfeat.bigfeat_base as bigfeat
import pickle
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import warnings
from sklearn.preprocessing import MinMaxScaler
import traceback


Import the dataset

In [3]:
# Paths to the pickle files
independent_path = '../Data/Cleaned Data/Independent_Data_dictionary.pkl'
dependent_path = '../Data/Cleaned Data/Dependent_Data_dictionary.pkl'

# Load the data
with open(independent_path, 'rb') as f:
    Independent_Data = pickle.load(f)

with open(dependent_path, 'rb') as f:
    Dependent_Data = pickle.load(f)

Encoding the variables

In [4]:
def encode_dataframes(data_dict):
    encoder = LabelEncoder()
    for key, df in data_dict.items():
        for col in df.select_dtypes(include=['object', 'category']).columns:
            df[col] = encoder.fit_transform(df[col])
    return data_dict

# Encode the dataframes in both dictionaries
Independent_Data_Encoded = encode_dataframes(Independent_Data)
Dependent_Data_Encoded = encode_dataframes(Dependent_Data)


Transforming the features

In [5]:
# Initialize cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Set names for datasets
regression_datasets = ['fri_c3_1000_50',
 'fri_c2_1000_25',
 'fri_c4_500_50',
 'fri_c4_1000_50',
 'fri_c1_1000_25',
 'fri_c1_500_50',
 'fri_c3_1000_25',
 'auto93',
 'pyrim',
 'autoPrice',
 'boston',
 'Concrete_Compressive_Strength',
 'Auto_MPG',
 'Forest Fires',
 'Servo',
 'Airfoil_Self_Noise',
 'Wine_Quality',
 'BodyFat',
 'California_Housing',
 'Quake']

#Initiate the model to normalize data
scaler = MinMaxScaler()

In [ ]:
# Suppress warnings for metrics that might be undefined for some classifiers
warnings.filterwarnings("ignore")


bf = bigfeat.BigFeat()

# Processing datasets
for dataset_name, X in Independent_Data_Encoded.items():
    if dataset_name in regression_datasets:
        pass
    else:
        try:
            fold_data = {}  # Dictionary to store fold data
            times = []
            print(f'Processing dataset: {dataset_name}')
            y = Dependent_Data_Encoded[dataset_name]

            fold_count = 1
            # Manually handling cross-validation to incorporate BigFeat
            for train_index, test_index in kf.split(X):
                fold_key = f'fold{fold_count}'
                fold_count += 1
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                
                # BigFeat feature engineering
                # Start timing for BigFeat
                start_time = time.time()
                res = bf.fit(X_train, y_train)
                # End timing after fitting the model
                transform_time = time.time() - start_time
                times.append(transform_time)

                X_train_transformed = pd.DataFrame(bf.transform(X_train))
                X_test_transformed = pd.DataFrame(bf.transform(X_test))

                # Apply normalization on each DataFrame
                X_train_transformed = pd.DataFrame(scaler.fit_transform(X_train_transformed), columns=X_train_transformed.columns, index=X_train_transformed.index)
                X_test_transformed = pd.DataFrame(scaler.transform(X_test_transformed), columns=X_test_transformed.columns, index=X_test_transformed.index)

                fold_data[fold_key] = {
                    'Training_Independent': X_train_transformed,
                    'Training_Dependent': y_train,
                    'Testing_Independent': X_test_transformed,
                    'Testing_Dependent': y_test,
                    'Timing': transform_time
                }

            # Serialize fold data to a .pkl file
            with open(f'../Data/BIGFEAT/{dataset_name}_fold_data.pkl', 'wb') as file:
                pickle.dump(fold_data, file)

        except Exception as e:
            print(f"Error processing {dataset_name}: {e}")
            traceback.print_exc()  # This will print the traceback of the exception
